Often data sources are incomplete, which means you will have missing data, you have 3 basic options for filling in missing data (you will personally have to make the decision for what is the right approach:


#### Just keep the missing data points.

#### Drop them missing data points (including the entire row)

#### Fill them in with some other value.


In [1]:
from pyspark.sql import SparkSession

# creating sparksession object
spark = SparkSession.builder.appName("handling_missing_data").getOrCreate()

24/03/19 22:37:40 WARN Utils: Your hostname, dhiraj resolves to a loopback address: 127.0.1.1; using 192.168.10.66 instead (on interface wlo1)
24/03/19 22:37:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/19 22:37:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Reading/Loading .csv file

In [2]:
df = spark.read.csv("../data/csv/containsnull.csv", header=True, inferSchema=True)

In [3]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### `Drop` the `missing` data

You can use the .na functions for missing data. 

The drop command has the following parameters:

df.na.drop(how='any', thresh=None, subset=None)


* param how: 'any' or 'all'.

    If 'any', drop a row if it contains any nulls.
    If 'all', drop a row only if all its values are null.


* param thresh: int, default None

    If specified, drop rows that have less than `thresh` non-null values.
    This overwrites the `how` parameter.

* param subset: 
    optional list of column names to consider.

#### Dropping `any row` that contains missing data

In [4]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



#### Dropping the row, `if row has not even 2 non-null values`

In [5]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



#### Dropping row, `if row has any null values`

In [6]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



#### Dropping row, `if row has all null values`

In [7]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### `Fill` the missing values

We can also fill the missing values with new values. 

If you have multiple nulls across multiple data types, Spark is actually smart enough to match up the data types. 

Watch below example:

In [8]:
df.na.fill('NEW VALUE').show()

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| NULL|
|emp2|NEW VALUE| NULL|
|emp3|NEW VALUE|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [9]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



#### BEST WAY:  Usually you should specify what columns you want to fill with the subset parameter.

In [10]:
df.na.fill('No Name',subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| NULL|
|emp2|No Name| NULL|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [11]:
df.na.fill(0, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



#### Best practice is to fill `null/nan` values with the `mean value` for the column.

In [12]:
from pyspark.sql.functions import mean

mean_val = df.select(mean(df['Sales'])).collect()

## Weird nested formatting of Row object
# mean_val[0][0]

In [13]:
mean_sales = mean_val[0][0]

In [14]:
# filling null values with `mean` values

df.na.fill(mean_sales, ["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### One linear code:

In [15]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

